In [64]:
import os
import cv2
import json
import numpy as np
from pathlib import Path
from random import randint
import matplotlib.pyplot as plt

In [65]:
class_files_root_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\labels")
json_root_path = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\annotations")
output_path = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\annotations_classes")
image_root_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\images")
segmentation_output_path = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\segmentation_data\\labels")

In [66]:
json_file_paths = list(json_root_path.glob("*.json"))

tooth_classes = {0: '11', 1: '12', 2: '13', 3: '14',4: '15', 5: '16', 6: '17', 7: '18', 8: '21', 9: '22', 10: '23', 11: '24', 12: '25', 13: '26', 14: '27', 15: '28', 16: '31', 17: '32', 18: '33', 19: '34', 20: '35', 21: '36', 22: '37', 23: '38', 24: '41', 25: '42', 26: '43', 27: '44', 28: '45', 29: '46', 30: '47', 31: '48'}

In [131]:
def get_json_file_data(json_file_path):
    with open(json_file_path, 'r') as json_file:
        return json.load(json_file)

def get_segmentation_polygon(json_file_path: os.PathLike):
    segment = []
    json_data = get_json_file_data(json_file_path)
    for tooth_data in json_data['annotations']:
        path = tooth_data['polygon']['path']
        segment.append([[point['x'], point['y']] for point in path])
    return segment

def segment_json_to_numpy(json_file_path:os.PathLike, get_image_shape:bool=True):
    json_data = get_json_file_data(json_file_path)
    bboxes = []
    for segment in json_data['annotations']:
        box = segment['bounding_box']
        bboxes.append([box['x'], box['y'], box['w'], box['h']])
    
    if get_image_shape:
        return np.array(bboxes), (json_data['image']['height'], json_data['image']['width'])


def adjust_polygon_to_bbox(polygon, bbox):
    x_min, y_min, w, h = bbox
    adjusted_polygon = [[point[0] - x_min, point[1] - y_min] for point in polygon]

    return adjusted_polygon


def draw_polygon_on_image(image, polygon, color=(255, 0, 0), thickness=2):
    points = np.array(polygon, np.int32)
    # points = points.reshape((-1, 1, 2))
    cv2.polylines(image, [points], isClosed=True, color=color, thickness=thickness)
    
    
def normalize_polygons(polygon:np.array, img_shape:tuple[int,int]):
    
    polygon = np.array(polygon)
    polygon[:, 0] /= img_shape[1]
    polygon[:, 1] /= img_shape[0]
    polygon[:, 0] = np.minimum(polygon[:, 0], 1)
    polygon[:, 1] = np.minimum(polygon[:, 1], 1)
    
    return polygon


def write_txt_file(normalized_polygon, txt_file_path):
    with open(txt_file_path, 'w') as f:
        line = '0 '
        f.write(line)
        for point in normalized_polygon:
            x, y = point
            line = f"{x} {y} "
            f.write(line)


In [ ]:
count = 0
for json_file_path in json_file_paths:
    count += 1
    if not json_file_path.exists():
        continue
    
    class_file_path = (class_files_root_path/json_file_path.stem).with_suffix(".txt")
    txt_file_path = (segmentation_output_path /json_file_path.stem).with_suffix(".txt")

    if not class_file_path.exists():
        continue
    
    image_path = (image_root_path/json_file_path.stem).with_suffix(".jpg")
    segmentation_boxes, img_shape = segment_json_to_numpy(json_file_path)
    polygons = get_segmentation_polygon(json_file_path)

    image = cv2.imread(str(image_path))
    for i, box in enumerate(segmentation_boxes):
        x, y, w, h = box.astype(int)
        cropped_roi = image[y: y + h, x: x + w]

        adjusted_polygon = adjust_polygon_to_bbox(polygons[i], box)
        
        normalized_polygon = normalize_polygons(adjusted_polygon, cropped_roi.shape)
        # print(normalized_polygon)
        cropped_roi_with_polygon = cropped_roi.copy()
        draw_polygon_on_image(cropped_roi_with_polygon, adjusted_polygon)

        plt.imshow(cv2.cvtColor(cropped_roi_with_polygon, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

    if count == 2:
        break

In [132]:
file_count = 1
count = 0
for json_file_path in json_file_paths:
    count += 1
    
    if not json_file_path.exists():
        continue

    if not class_file_path.exists():
        continue
    
    image_path = (image_root_path/json_file_path.stem).with_suffix(".jpg")
    segmentation_boxes, img_shape = segment_json_to_numpy(json_file_path)
    polygons = get_segmentation_polygon(json_file_path)


    image = cv2.imread(str(image_path))
    for i, box in enumerate(segmentation_boxes):
        x, y, w, h = box.astype(int)
        
        if w <= 0 or h <= 0:
            print(f"Invalid bounding box dimensions: {box}")
            continue

        cropped_roi = image[y: y + h, x: x + w]
        cropped_roi_shape = cropped_roi.shape
        # print(cropped_roi_shape)
        if cropped_roi.size == 0:
            print(f"Empty cropped ROI for bounding box: {box}")
            continue
        
        txt_file_path = f"C:\\Users\\AMET\\DocGaid\\yolov8\\data\\single_tooth\\labels\\train2\\{file_count}.txt"
        adjusted_polygon = adjust_polygon_to_bbox(polygons[i], box)
        
        normalized_polygon = normalize_polygons(adjusted_polygon, cropped_roi.shape)
        np.set_printoptions(precision=6, suppress=True)
        print(normalized_polygon)

        cropped_image_path = f"C:\\Users\\AMET\\DocGaid\\yolov8\\data\\single_tooth\\images\\train2\\{file_count}.jpg"
        cv2.imwrite(cropped_image_path, cropped_roi)

        write_txt_file(normalized_polygon, txt_file_path)
        file_count+=1


[[0.11189  0.011198]
 [0.091024 0.004219]
 [0.067559 0.000495]
 [0.051496 0.      ]
 [0.031417 0.00138 ]
 [0.015669 0.003854]
 [0.002047 0.007813]
 [0.000315 0.011198]
 [0.       0.024115]
 [0.04126  0.261693]
 [0.127323 0.588177]
 [0.167874 0.763021]
 [0.231181 0.925208]
 [0.264803 0.954479]
 [0.303701 0.972578]
 [0.35685  0.989609]
 [0.431654 1.      ]
 [0.574803 1.      ]
 [0.715748 0.99138 ]
 [0.891575 0.974505]
 [0.929764 0.97    ]
 [0.962205 0.956641]
 [0.98937  0.938359]
 [1.       0.914089]
 [0.995669 0.824557]
 [0.969134 0.747292]
 [0.864724 0.64487 ]
 [0.185354 0.058594]]
[[0.181748 0.06569 ]
 [0.075922 0.010337]
 [0.049709 0.003131]
 [0.020388 0.      ]
 [0.008252 0.003131]
 [0.       0.017306]
 [0.01068  0.034377]
 [0.025243 0.058855]
 [0.03835  0.094646]
 [0.051553 0.187946]
 [0.079903 0.440101]
 [0.099417 0.620168]
 [0.088058 0.679798]
 [0.05699  0.7367  ]
 [0.001553 0.820404]
 [0.007184 0.835118]
 [0.021165 0.886566]
 [0.030874 0.900067]
 [0.05534  0.925589]
 [0.089126 0

KeyError: 'polygon'